### doc_topics
ESTABELECE RELAÇÕES ENTRE DOCUMENTOS E TÓPICOS NA BASE DE DADOS NO MYSQL. TRABALHAMOS COM O ACERVO **ANTONIO AZEREDO DA SILVEIRA, MINISTÉRIO DAS RELAÇÕES EXTERIORES**.

### Observações

#### Sobre como cria tabela CSV com limite documentos por tópico

In [16]:
import nltk
import os
import codecs
import string
import matplotlib.pyplot as plt
import matplotlib as mpl
from gensim import corpora, models, similarities #Latent Dirichlet Allocation implementation with Gensim
import pyLDAvis
import pyLDAvis.gensim
from IPython.display import clear_output

import getpass
from sshtunnel import SSHTunnelForwarder
import pymysql

import pickle

In [17]:
outputs = "../outputs/"

#### Carrega o treinamento

In [18]:
#filename = 'C:/Users/marcelo.ribeiro/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Azeredo Papers/Processamento/text-learning-tools/outputs/trainedLDA.sav'
file_corpus = '../outputs/LDAcorpus.sav'
file_dictionary = '../outputs/LDAdictionary.sav'
corpus = pickle.load(open(file_corpus, 'rb'))
dictionary = pickle.load(open(file_dictionary, 'rb'))

In [19]:
len(corpus)

10

In [20]:
%time lda = models.LdaModel(corpus, num_topics=50, id2word=dictionary, passes=10)

Wall time: 582 ms


In [24]:
lda.print_topics(-1, num_words=10)

[(0,
  '0.002*et + 0.002*brbsilid + 0.002*recebidos + 0.002*souto + 0.002*dq + 0.002*romeu + 0.002*1-45 + 0.002*assinatura + 0.002*iÃ¢so + 0.002*2/4'),
 (1,
  '0.002*xx>4Ã¡iÃ£fkk + 0.002*col + 0.002*generosas + 0.002*ma + 0.002*wladimir + 0.002*caros + 0.002*lvei + 0.002*iaiÃ¡ + 0.002*velhos + 0.002*chefia'),
 (2,
  '0.002*de + 0.002*gb + 0.002*rio + 0.002*professor + 0.002*abraco + 0.002*generosas + 0.002*bsb + 0.002*exterior + 0.002*prezado + 0.002*pt'),
 (3,
  '0.002*xx>4Ã¡iÃ£fkk + 0.002*col + 0.002*generosas + 0.002*ma + 0.002*wladimir + 0.002*caros + 0.002*lvei + 0.002*iaiÃ¡ + 0.002*velhos + 0.002*chefia'),
 (4,
  '0.002*xx>4Ã¡iÃ£fkk + 0.002*col + 0.002*generosas + 0.002*ma + 0.002*wladimir + 0.002*caros + 0.002*lvei + 0.002*iaiÃ¡ + 0.002*velhos + 0.002*chefia'),
 (5,
  '0.002*xx>4Ã¡iÃ£fkk + 0.002*col + 0.002*generosas + 0.002*ma + 0.002*wladimir + 0.002*caros + 0.002*lvei + 0.002*iaiÃ¡ + 0.002*velhos + 0.002*chefia'),
 (6,
  '0.002*xx>4Ã¡iÃ£fkk + 0.002*col + 0.002*generosas + 0.0

In [25]:
lda.show_topics(num_topics=-1)

[(0,
  '0.002*et + 0.002*brbsilid + 0.002*recebidos + 0.002*souto + 0.002*dq + 0.002*romeu + 0.002*1-45 + 0.002*assinatura + 0.002*iÃ¢so + 0.002*2/4'),
 (1,
  '0.002*xx>4Ã¡iÃ£fkk + 0.002*col + 0.002*generosas + 0.002*ma + 0.002*wladimir + 0.002*caros + 0.002*lvei + 0.002*iaiÃ¡ + 0.002*velhos + 0.002*chefia'),
 (2,
  '0.002*de + 0.002*gb + 0.002*rio + 0.002*professor + 0.002*abraco + 0.002*generosas + 0.002*bsb + 0.002*exterior + 0.002*prezado + 0.002*pt'),
 (3,
  '0.002*xx>4Ã¡iÃ£fkk + 0.002*col + 0.002*generosas + 0.002*ma + 0.002*wladimir + 0.002*caros + 0.002*lvei + 0.002*iaiÃ¡ + 0.002*velhos + 0.002*chefia'),
 (4,
  '0.002*xx>4Ã¡iÃ£fkk + 0.002*col + 0.002*generosas + 0.002*ma + 0.002*wladimir + 0.002*caros + 0.002*lvei + 0.002*iaiÃ¡ + 0.002*velhos + 0.002*chefia'),
 (5,
  '0.002*xx>4Ã¡iÃ£fkk + 0.002*col + 0.002*generosas + 0.002*ma + 0.002*wladimir + 0.002*caros + 0.002*lvei + 0.002*iaiÃ¡ + 0.002*velhos + 0.002*chefia'),
 (6,
  '0.002*xx>4Ã¡iÃ£fkk + 0.002*col + 0.002*generosas + 0.0

In [26]:
topics_matrix = lda.show_topics(formatted=False, num_words=20, num_topics=-1)
topics_matrix[34]

(34,
 [('de', 0.095894873281481025),
  ('uma', 0.028286303038841762),
  ('no', 0.028278638610228946),
  ('com', 0.022645039227184433),
  ('as', 0.022617980635262738),
  ('rio', 0.011388078284857151),
  ('sobre', 0.011386197592867214),
  ('mais', 0.011384692279408996),
  ('que', 0.011381597228523164),
  ('depois', 0.011381528526713616),
  ('da', 0.011379747928829143),
  ('para', 0.011379453109493228),
  ('em', 0.0113771155023293),
  ('sua', 0.0059419612451074152),
  ('negociaÃ§Ãµes', 0.0057535142267533465),
  ('terezinha', 0.0057535093284843729),
  ('exteikm', 0.0057528049819189101),
  ('presidente', 0.0057520942016335824),
  ('expedito', 0.0057518043713269858),
  ('aÃ£', 0.0057516404292567783)])

#### Visualiza os tópicos com o PyLDAvis

In [27]:
# https://cran.r-project.org/web/packages/LDAvis/vignettes/details.pdf
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda, corpus, dictionary)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
43     15.801235        1       1  0.169367 -0.171260
28      8.063198        1       2  0.108518 -0.071393
15      6.230020        1       3  0.133751  0.011469
10      5.723327        1       4  0.223910 -0.068073
33      5.320680        1       5  0.138644  0.005871
39      4.523966        1       6  0.239441 -0.075219
24      4.505990        1       7  0.204625 -0.104404
3       4.478655        1       8  0.267187 -0.053680
12      3.274494        1       9  0.160672 -0.161245
25      3.225167        1      10  0.228037 -0.195689
31      3.150813        1      11  0.160008 -0.133729
19      3.123075        1      12  0.158167  0.003757
5       3.002638        1      13  0.084252  0.238428
8       2.793116        1      14  0.185341 -0.091039
42      2.766307        1      15  0.231679 -0.028476
45      2.543985        1      16  0.183784 -0.084310
44      2.282358        1      17  0.009524  0.067968
20      2.280163        1      18  0.185816  0.037328
18      2.147396        1      19  0.160612  0.215994
14      2.089084        1      20  0.035055  0.109330
32      2.061049        1      21  0.087839  0.283055
46      1.713549        1      22  0.165685 -0.020676
22      1.551292        1      23  0.127357  0.164925
1       1.169768        1      24  0.188190  0.008596
47      0.835747        1      25  0.041872  0.250402
13      0.795407        1      26 -0.000951  0.108071
16      0.616201        1      27  0.096668 -0.064340
37      0.473937        1      28 -0.071688  0.064059
29      0.390046        1      29  0.037659 -0.115942
17      0.365773        1      30 -0.069060  0.085657
9       0.335384        1      31 -0.089917  0.067981
36      0.319185        1      32 -0.110530 -0.001385
49      0.311578        1      33 -0.033291  0.024062
35      0.305950        1      34 -0.111676  0.084852
27      0.302486        1      35 -0.105641  0.068946
21      0.258622        1      36 -0.099639  0.136976
6       0.197603        1      37 -0.130503  0.113160
41      0.165281        1      38 -0.173441  0.051103
0       0.102321        1      39 -0.215989 -0.036774
4       0.071136        1      40 -0.238019 -0.046143
2       0.070813        1      41 -0.201428 -0.047594
34      0.067471        1      42 -0.240236 -0.049194
7       0.054510        1      43 -0.255408 -0.054703
38      0.041138        1      44 -0.252506 -0.065800
48      0.032260        1      45 -0.265843 -0.067642
40      0.020359        1      46 -0.269444 -0.075072
11      0.016744        1      47 -0.269405 -0.075185
30      0.012571        1      48 -0.270047 -0.079371
23      0.008139        1      49 -0.269414 -0.081966
26      0.008009        1      50 -0.269585 -0.081687, topic_info=      Category          Freq             Term         Total  loglift  logprob
term                                                                         
27915  Default  18935.000000           senhor  18935.000000  30.0000  30.0000
77619  Default  21130.000000         ministro  21130.000000  29.0000  29.0000
42128  Default  12800.000000       excelência  12800.000000  28.0000  28.0000
95240  Default  13482.000000        telegrama  13482.000000  27.0000  27.0000
32676  Default  21432.000000       presidente  21432.000000  26.0000  26.0000
58370  Default  16386.000000       exteriores  16386.000000  25.0000  25.0000
10331  Default  23176.000000          governo  23176.000000  24.0000  24.0000
7410   Default   7935.000000         silveira   7935.000000  23.0000  23.0000
59893  Default  24019.000000         relações  24019.000000  22.0000  22.0000
26085  Default  48420.000000           brasil  48420.000000  21.0000  21.0000
95187  Default   8517.000000        chanceler   8517.000000  20.0000  20.0000
14961  Default  11634.000000       embaixador  11634.000000  19.0000  19.0000
393    Default  12142.000000        república  12142.0000

In [ ]:
data_ldavis = pyLDAvis.gensim.prepare(lda, corpus, dictionary)
pyLDAvis.save_html(data_ldavis, os.path.join(outputs,'pyldavis_output.html'))

#### Gera a distribuição de tópicos para um novo documento
Alimenta a base mysql com dados de score (relação) entre tópicos e documentos

Gerando a distribuição de tópicos para um novo documento:

In [ ]:
count = 0
#percentil = int(len(cursor.rowcount/100)

pass_sshkey = getpass.getpass()
pass_mysql = getpass.getpass()
with SSHTunnelForwarder(('200.20.164.147', 22),
                        #ssh_private_key = "C:/Users/marcelo.ribeiro/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Remoto/marcelo_priv_rsa",
                        ssh_private_key = '/home/rsouza/.ssh/id_rsa',
                        ssh_private_key_password = pass_sshkey,
                        #ssh_username="marcelobribeiro",
                        ssh_username="rsouza",
                        remote_bind_address=('127.0.0.1', 3306)) as server:
    
    conn = pymysql.connect(host='localhost', 
                           port=server.local_bind_port, 
                           #user='marcelobribeiro', 
                           user='rsouza',
                           passwd=pass_mysql,
                           db='CPDOC_AS',
                           use_unicode=True, 
                           charset="utf8")
    cur = conn.cursor()
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    insere dados na base mysql
    captura documentos da tabela docs, transfere para a tabela topic-doc e insere dados de tópicos
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    
    cur.execute("DROP TABLE IF EXISTS topic_doc_lang_filter")
    cur.execute('''CREATE TABLE IF NOT EXISTS topic_doc_lang_filter 
               (doc_id VARCHAR(31), topic_id smallint(6), topic_score FLOAT
               DEFAULT NULL)
               ENGINE=MyISAM DEFAULT CHARSET='utf8';''')

    cur.execute("SELECT id,body FROM CPDOC_AS.docs WHERE main_language = 1 AND readability != 'low' ") # filtra textos
    data = cur.fetchall()
    for row in data:
        count += 1 # for break event
        if row is None: break
        text =  row[1]
        text = text.split()
        text_bow = dictionary.doc2bow(text)
        #print(lda[text_bow])
        score_list = lda[text_bow]
        doc_id = row[0]
        for score in score_list:
            topic_id = str(score[0])
            topic_score = str(score[1])
            query = "INSERT INTO topic_doc_lang_filter VALUES (%s, %s, %s)"
            cur.execute(query, (doc_id, topic_id, topic_score))
        #print(doc_id, topic_id, topic_score)   
        cur.execute("ALTER TABLE CPDOC_AS.topic_doc_lang_filter ORDER BY topic_id ASC, topic_score DESC")
        
